# Cloudless images from Sentinel HUB

Find cloudless images for a selected AOI in a selected time frame.

In [1]:
from sentinelhub import SHConfig

config = SHConfig()

#config.instance_id = '<your instance id>'
#config.sh_client_id = '<your client id>'
#config.sh_client_secret = '<your client secret>'

config

SHConfig(
  instance_id='CEOIS2',
  sh_client_id='d680e4fa-c777-4c98-ae42-ce515fc389ca',
  sh_client_secret='1n*Pe/M9|/gDC9ctn.)a6zN{y2j+h-aSzp{z*Xyk',
  sh_base_url='https://services.sentinel-hub.com',
  geopedia_wms_url='https://service.geopedia.world',
  geopedia_rest_url='https://www.geopedia.world/rest',
  aws_access_key_id='',
  aws_secret_access_key='',
  aws_metadata_url='https://roda.sentinel-hub.com',
  aws_s3_l1c_bucket='sentinel-s2-l1c',
  aws_s3_l2a_bucket='sentinel-s2-l2a',
  opensearch_url='http://opensearch.sentinel-hub.com/resto/api/collections/Sentinel2',
  max_wfs_records_per_query=100,
  max_opensearch_records_per_query=500,
  max_download_attempts=4,
  download_sleep_time=5,
  download_timeout_seconds=120,
  number_of_download_processes=1
)

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [52]:
# import os
# import datetime
# import numpy as np
# import matplotlib.pyplot as plt
# from utils import plot_image
# import osmnx as ox
# import glob
# import rasterio
# import rasterio.plot

from sentinelhub import CRS, BBox, bbox_to_dimensions, DataCollection, \
    SentinelHubCatalog, filter_times, \
    SentinelHubRequest, MimeType, SentinelHubDownloadClient

import datetime as dt
#import tarfile
from os.path import join, dirname
from os import rename
from shutil import rmtree

## Prepare input data

In [4]:
# Data collection
data_collection = DataCollection.SENTINEL2_L2A

# Resolution (pixel size)
resolution = 10

# CRS for sentinelhub query getattr() assigns module CRS with UTM_### method ==> CRS.UTM_36N
utm_zone = "36N"
sample_crs = getattr(CRS, f"UTM_{utm_zone}")
# List of supported CRSs: https://docs.sentinel-hub.com/api/latest/api/process/crs/

# ------------------------------------------------------------------------------------------------

## Build bbox from centrepoint + distance
# centrepoint
utm_x = 512817
utm_y = 269983
# AOI size in meters (x, y)
aoi_size = (5120, 5120)

# Round UTM coords to nearest 10 (so the raster will look nice in UTM - "target aligned pixels")
x_y = (round(utm_x, -1), round(utm_y, -1)) 

bbox_utm = [x_y[0] - aoi_size[0]/2,
            x_y[1] - aoi_size[1]/2,
            x_y[0] + aoi_size[0]/2,
            x_y[1] + aoi_size[1]/2]


# ## Use this to create a bbox directly:
# bbox_utm = [510260, 267420, 515380, 272540]

# ------------------------------------------------------------------------------------------------

# Prepare some data for sentinelhub
sample_bbox = BBox(bbox=bbox_utm, crs=sample_crs)
sample_size = bbox_to_dimensions(sample_bbox, resolution=resolution)
print(sample_size)
print(sample_bbox)
print(sample_crs)

(512, 512)
510260.0,267420.0,515380.0,272540.0
EPSG:32636


## Search for all available images in a time frame

Kako pa dobimo vse posnetke (vse datume) znotraj enega četrtletja?

In [ ]:
%%time
# Sarch catalog for all images with bbox in a time interval
catalog = SentinelHubCatalog(config=config)

# SET TIME INTERVAL
time_interval = '2018-01-01', '2019-01-01'

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L2A,
    bbox=sample_bbox,
    time=time_interval,
    fields={
        "include": [
            "id",
            "properties.datetime",
            "properties.eo:cloud_cover"
        ],
        "exclude": []
    }

)

results = list(search_iterator)
print('Total number of results:', len(list(search_iterator)))
print(results[:4])

# Many timestamps differ only for a few seconds. That is because they are from
# tiles in the same orbit acquisition. We want to join them together in a single timestamp.

time_difference = dt.timedelta(hours=1)
all_timestamps = search_iterator.get_timestamps()

unique_acquisitions = filter_times(all_timestamps, time_difference)
print(f"Number of unique acquisitions: {len(unique_acquisitions)}")

Total number of results: 106
[{'id': 'S2B_MSIL2A_20181231T080329_N0211_R035_T36NWH_20181231T095733', 'properties': {'datetime': '2018-12-31T08:19:26Z', 'eo:cloud_cover': 5.15}}, {'id': 'S2A_MSIL2A_20181226T080331_N0211_R035_T36NWH_20181226T104208', 'properties': {'datetime': '2018-12-26T08:19:22Z', 'eo:cloud_cover': 1.03}}, {'id': 'S2B_MSIL2A_20181221T080329_N0211_R035_T36NWH_20181221T141004', 'properties': {'datetime': '2018-12-21T08:19:24Z', 'eo:cloud_cover': 2.16}}, {'id': 'S2A_MSIL2A_20181216T080331_N0211_R035_T36NWH_20181216T102435', 'properties': {'datetime': '2018-12-16T08:19:20Z', 'eo:cloud_cover': 8.69}}]
Number of unique acquisitions: 72
Wall time: 1.12 s


## Filter by cloud cover

### Download all CLMs

In [7]:
%%time

# Create a Process API request for each acquisition:
cm_evalscript = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["CLM"]
            }],
            output: {
                bands: 1
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.CLM];
    }
"""

# Requests will be appenede to this list
process_requests = []

for timestamp in unique_acquisitions:
    request = SentinelHubRequest(
        evalscript=cm_evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=data_collection,
                time_interval=(timestamp - time_difference, timestamp + time_difference)
            )
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.PNG)
        ],
        bbox=sample_bbox,
        size=sample_size,
        config=config
    )
    process_requests.append(request)
    

# In order to efficiently download data for all requests in parallel, we extract download information and pass it to a download client.
client = SentinelHubDownloadClient(config=config)
download_requests = [request.download_list[0] for request in process_requests]
data = client.download(download_requests)

print(f"Downloaded: {len(data)}")

Downloaded: 72
Wall time: 4.81 s


### Filter by cloud cover

In [57]:
# Select threshold (e.g. 0.05 for less or equal than 5%)
threshold = 0
# Calculat cloud ratio for all images
cc_ratio = [(cc_image == 255).sum() / cc_image.size for cc_image in data]
# Select those below the threshold
lt5_cc = [(i, cc) for i, cc in zip(unique_acquisitions, cc_ratio) if cc <= threshold]

print(len(lt5_cc))
lt5_cc

29


[(datetime.datetime(2018, 1, 5, 8, 18, 52, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 1, 10, 8, 15, 26, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 1, 15, 8, 17, 55, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 1, 25, 8, 8, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 1, 30, 8, 11, 4, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 2, 4, 8, 14, 15, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 2, 9, 8, 19, 3, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 2, 14, 8, 18, 8, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 3, 11, 8, 14, 46, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 5, 15, 8, 9, 13, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 6, 9, 8, 3, 43, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 6, 14, 8, 18, 11, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 6, 29, 8, 18, 42, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 7, 14, 8, 11, 56, tzinfo=tzutc()), 0.0),
 (datetime.datetime(2018, 7, 24, 8, 16, 36, tzinfo=tzutc()), 0.0),
 (datetim

## Download selected images (num)

Evalscript parameters: [https://docs.sentinel-hub.com/api/latest/evalscript/v3/#parameters](https://docs.sentinel-hub.com/api/latest/evalscript/v3/#parameters)

In [69]:
%%time

# Double curved brackets, so we can insert variables using .format() method where single brackets are present
my_evalscript = """
    //VERSION=3

    function setup() {{
        return {{
            input: [{{
                bands: ["B02","B03","B04"],
                units: "reflectance"  // "DN" for Digital Numbers, "reflectance" for normalised
            }}],
            output: {{
                id: "{0}",
                bands: 3,
                sampleType: "FLOAT32"  // "UINT16" for "DN", "FLOAT32" for "reflectance" 
            }}
        }};
    }}
    
    //NOT USED IF ONLY ONE TIFF IS DEFINED IN RESPONSES (uncomment in the SentinelHubRequest below)
    function updateOutputMetadata(scenes, inputMetadata, outputMetadata) {{
        outputMetadata.userData = {{ "norm_factor":  inputMetadata.normalizationFactor }}
    }}

    function evaluatePixel(sample) {{
        return [sample.B04, sample.B03, sample.B02];
    }}
"""

data_folder = ".\\downloaded"

process_requests = []
save_locations = []
for timestamp, _ in lt5_cc[:3]:
    # Insert custom name for output file into evalscript
    my_string = timestamp.strftime("S2_%Y%d%mT%H%M%S_cloudless")
    evalscript = my_evalscript.format(my_string)
    
    # Define custom name for correct file naming
    my_folder = join(data_folder, my_string)
    
    request = SentinelHubRequest(
        data_folder=my_folder,
        evalscript=evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=data_collection,
                time_interval=(timestamp - time_difference, timestamp + time_difference)
            )
        ],
        responses=[
            SentinelHubRequest.output_response(my_string, MimeType.TIFF),
            #SentinelHubRequest.output_response("userdata", MimeType.JSON)
        ],
        bbox=sample_bbox,
        size=sample_size,
        config=config
    )
    process_requests.append(request)
    save_locations.append(join(my_folder, request.get_filename_list()[0]))


# Download files
download_requests = [request.save_data(raise_download_errors=True) for request in process_requests]

# Rename filest
for file in save_locations:
    target_name = dirname(dirname(file))
    rename(file, target_name + ".tiff")
    # Remove leftover folder
    rmtree(target_name)

# # Now extract all the files (only if more than one output was defined)
# abc = [request.get_filename_list()[0] for request in process_requests]
# for image in abc:
#     tar_path = join(data_folder, image)
#     with tarfile.open(tar_path) as my_tar:
#         my_tar.extractall(data_folder)
#     rmtree(dirname(tar_path))

Wall time: 5.39 s


## Visualize results in Jupyter Notebook

TBA

## Download numpy and save GeoTIFF with rasteri

TBA